<a href="https://colab.research.google.com/github/Cassiosa/InvestWise/blob/main/InvestWise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O projeto será dividido em três etapas:

Coleta e Tratamento de Dados:

Utilizaremos APIs financeiras (como a yfinance para obter dados históricos de ações, ETFs, criptomoedas, etc.).
Permitiremos que os usuários escolham ativos para monitorar.
Faremos um tratamento básico dos dados para análise.

Análise e Previsão:

Implementaremos um modelo preditivo para prever o desempenho futuro dos ativos com base em dados históricos.
Usaremos modelos de Machine Learning, como ARIMA ou Prophet, para prever rentabilidade.
Calcularemos métricas de risco e retorno, como Sharpe Ratio, Drawdown, e volatilidade.

Visualização e Insights:

Criaremos visualizações em gráficos para ajudar o usuário a entender o desempenho e a previsão dos ativos.
Forneceremos insights acionáveis, como recomendações de diversificação de portfólio.

In [1]:
!pip install pandas numpy matplotlib seaborn yfinance scikit-learn prophet plotly

In [30]:
#Obtendo dados financeiros usando a biblioteca yfinance
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Função para coletar dados históricos de um ativo
def obter_dados_ativo(ticker, periodo='5y'):
    ativo = yf.Ticker(ticker)
    dados = ativo.history(period=periodo)
    dados.reset_index(inplace=True)
    dados['Ticker'] = ticker
    return dados

# Exemplo de uso
tickers = ['AAPL', 'MSFT', 'TSLA']  # Usuário pode adicionar mais ativos
df = pd.concat([obter_dados_ativo(ticker) for ticker in tickers], ignore_index=True)

# Mostrar uma amostra dos dados
print(df.head())

                       Date       Open       High        Low      Close  \
0 2019-11-18 00:00:00-05:00  64.422104  64.817169  64.041588  64.737190   
1 2019-11-19 00:00:00-05:00  64.931091  64.955330  64.322746  64.540878   
2 2019-11-20 00:00:00-05:00  64.359093  64.489967  63.113304  63.789520   
3 2019-11-21 00:00:00-05:00  63.910725  63.988285  63.302372  63.503544   
4 2019-11-22 00:00:00-05:00  63.644085  63.787083  63.219936  63.447765   

      Volume  Dividends  Stock Splits Ticker  
0   86703200        0.0           0.0   AAPL  
1   76167200        0.0           0.0   AAPL  
2  106234400        0.0           0.0   AAPL  
3  121395200        0.0           0.0   AAPL  
4   65325200        0.0           0.0   AAPL  


In [23]:
!pip install prophet matplotlib

In [28]:
#Usando o Prophet do Facebook para prever preços futuros
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
import yfinance as yf

def obter_dados_ativo(ticker, periodo='5y'):
    """Função para coletar dados históricos de um ativo"""
    ativo = yf.Ticker(ticker)
    dados = ativo.history(period=periodo)
    dados.reset_index(inplace=True)
    dados['Ticker'] = ticker
    return dados

def prever_rentabilidade(df, ticker):
    """Função para prever rentabilidade futura de um ativo"""
    # Filtrar o DataFrame pelo ticker desejado e preparar os dados
    df_ticker = df[df['Ticker'] == ticker][['Date', 'Close']].dropna()
    df_ticker.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)

    # Verificar se há dados suficientes para realizar a previsão
    if df_ticker.shape[0] < 10:
        print("Dados insuficientes para previsão.")
        return

    # Inicializar e treinar o modelo Prophet
    modelo = Prophet()
    modelo.fit(df_ticker)

    # Criar um DataFrame futuro para a previsão (próximos 180 dias)
    futuro = modelo.make_future_dataframe(periods=180)
    previsao = modelo.predict(futuro)

    # Plotar a previsão
    plt.figure(figsize=(10, 6))
    modelo.plot(previsao)
    plt.title(f'Previsão de Rentabilidade para {ticker}')
    plt.xlabel('Data')
    plt.ylabel('Preço Fechamento')
    plt.grid(True)
    plt.show()

    return previsao

# Exemplo de uso
# Coletar dados históricos para o ativo AAPL
df = obter_dados_ativo('AAPL')
prever_rentabilidade(df, 'AAPL')



ValueError: Column ds has timezone specified, which is not supported. Remove timezone.

In [27]:
#Análise de Risco e Retorno
def calcular_metricas(df, ticker):
    df_ticker = df[df['Ticker'] == ticker]
    df_ticker['Retorno'] = df_ticker['Close'].pct_change()

    retorno_medio = df_ticker['Retorno'].mean()
    volatilidade = df_ticker['Retorno'].std()
    sharpe_ratio = retorno_medio / volatilidade

    print(f"Métricas para {ticker}:")
    print(f"Retorno Médio: {retorno_medio:.4f}")
    print(f"Volatilidade: {volatilidade:.4f}")
    print(f"Sharpe Ratio: {sharpe_ratio:.4f}")

# Exemplo de uso
calcular_metricas(df, 'AAPL')

Métricas para AAPL:
Retorno Médio: 0.0012
Volatilidade: 0.0200
Sharpe Ratio: 0.0603


In [10]:
#Visualização e Insights com Plotly
import plotly.express as px

def visualizar_dados(df, ticker):
    df_ticker = df[df['Ticker'] == ticker]

    fig = px.line(df_ticker, x='Date', y='Close', title=f'Preço do {ticker} ao longo do tempo')
    fig.update_layout(xaxis_title='Data', yaxis_title='Preço de Fechamento')
    fig.show()

# Exemplo de uso
visualizar_dados(df, 'MSFT')

Conclusão

Com essa solução:

Monitoramos ativos em tempo real.
Prevemos rentabilidade futura para ajudar na tomada de decisões.
Fornecemos insights baseados em métricas financeiras.

Próximos Passos Detalhados

Adicionar Suporte a Diferentes Classes de Ativos

Objetivo: Diversificar os tipos de investimentos analisados para atender diferentes perfis de investidores, como ações, criptomoedas, fundos imobiliários (REITs), ETFs, etc.

Implementação:

Utilizar APIs como CoinGecko ou Binance API para dados de criptomoedas.
Usar a Alpha Vantage API para obter informações sobre ETFs e REITs.
Expandir a função obter_dados_ativo() para lidar com múltiplas APIs, dependendo do tipo de ativo selecionado.

In [22]:
import requests

def obter_dados_cripto(crypto_id, dias=365):
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
    params = {'vs_currency': 'usd', 'days': dias}
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data['prices'], columns=['Date', 'Close'])
    df['Date'] = pd.to_datetime(df['Date'], unit='ms')
    return df

# Exemplo de uso
df_crypto = obter_dados_cripto('bitcoin')
print(df_crypto.head())

        Date         Close
0 2023-11-20  37413.994608
1 2023-11-21  37489.298471
2 2023-11-22  35965.360361
3 2023-11-23  37464.832932
4 2023-11-24  37293.316127


Análise de Sentimento com Base em Notícias

Objetivo: Prever movimentos de mercado utilizando o sentimento de notícias financeiras e postagens em redes sociais.

Implementação:

Utilizar APIs como NewsAPI, Twitter API ou Finviz para coletar notícias e tweets relacionados a investimentos.

Utilizar técnicas de Processamento de Linguagem Natural (NLP), como VADER, TextBlob ou Hugging Face Transformers, para realizar análise de sentimento.

Integrar a análise de sentimento aos modelos preditivos para ajustar previsões com base nas tendências atuais.

In [21]:
from textblob import TextBlob
import requests

def obter_sentimento_noticias(query):
    url = f"https://newsapi.org/v2/everything?q={query}&apiKey=YOUR_API_KEY"
    response = requests.get(url)
    noticias = response.json()['articles']
    sentimentos = []
    for noticia in noticias:
        analise = TextBlob(noticia['title'])
        sentimentos.append(analise.sentiment.polarity)
    return np.mean(sentimentos)

# Exemplo de uso
sentimento_apple = obter_sentimento_noticias("Apple")
print(f"Sentimento para Apple: {sentimento_apple}")


KeyError: 'articles'

Sistema de Alertas com Base em KPIs

Objetivo: Notificar o usuário quando certos KPIs, como rentabilidade, volatilidade ou drawdown, atingirem um limite pré-determinado.

Implementação:
Utilizar a biblioteca smtplib para enviar alertas por email.
Usar Pushbullet API ou Twilio para enviar notificações em tempo real no celular.
Permitir que o usuário defina os KPIs e limites personalizados no dashboard.

In [20]:
import smtplib
from email.message import EmailMessage

def enviar_alerta_email(email_destino, assunto, mensagem):
    msg = EmailMessage()
    msg['Subject'] = assunto
    msg['From'] = 'seu_email@gmail.com'
    msg['To'] = email_destino
    msg.set_content(mensagem)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login('seu_email@gmail.com', 'sua senha')
        smtp.send_message(msg)

# Exemplo de uso
enviar_alerta_email('usuario@example.com', 'Alerta de Investimento',
                    'O ativo AAPL atingiu o limite de volatilidade!')


SMTPAuthenticationError: (534, b'5.7.9 Application-specific password required. For more information, go to\n5.7.9  https://support.google.com/mail/?p=InvalidSecondFactor a1e0cc1a2514c-856dfba28b6sm1205842241.2 - gsmtp')

Escalar para uma Aplicação Web com Streamlit ou Flask

Objetivo: Facilitar o uso do sistema para usuários leigos, transformando o projeto em um dashboard interativo.

Implementação:
Usar Streamlit para criar um dashboard visual e interativo onde o usuário pode selecionar ativos, visualizar gráficos e receber insights.
Alternativamente, usar Flask para criar uma aplicação web mais customizável.
Integrar o sistema de alertas, visualizações e previsões dentro da interface da aplicação.

In [15]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [16]:
import streamlit as st
import plotly.express as px

st.title("Dashboard de Investimentos")
ticker = st.text_input("Digite o Ticker do Ativo", "AAPL")

if st.button("Analisar"):
    dados = obter_dados_ativo(ticker)
    fig = px.line(dados, x='Date', y='Close', title=f'Preço de {ticker}')
    st.plotly_chart(fig)

# Sistema de alertas
limite_volatilidade = st.slider("Definir Limite de Volatilidade", 0.0, 0.1, 0.05)
volatilidade = calcular_metricas(dados, ticker)[1]
if volatilidade > limite_volatilidade:
    st.warning(f"Atenção: Volatilidade do {ticker} excedeu o limite!")


2024-11-18 18:28:20.477 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 18:28:20.583 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-11-18 18:28:20.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 18:28:20.588 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 18:28:20.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 18:28:20.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 18:28:20.596 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 18:28:20.599 Session state does not 

NameError: name 'dados' is not defined

Conclusão
Com esses aprimoramentos:

O sistema se torna mais robusto ao lidar com diferentes tipos de ativos.
A análise de sentimento traz um componente qualitativo para prever movimentos de mercado.

O sistema de alertas torna o monitoramento mais proativo.
Transformar o projeto em uma aplicação web torna-o mais acessível e intuitivo, especialmente para iniciantes.

Esses passos irão expandir significativamente o impacto do projeto, tornando-o uma ferramenta completa para investidores iniciantes e experientes. Se precisar de suporte adicional em alguma dessas etapas, estou aqui para ajudar! 🚀📊